In [ ]:
# install Bonito last version and Pydrive
# if the cell is run and a numpy warning pops up, restart kernel and run again the cell
# source: https://github.com/nanoporetech/bonito/blob/v0.4.0/notebooks/bonito-train.ipynb

!pip install -q ont-bonito
!pip install -U -q PyDrive
!pip install -q tensorly
!pip install -q tensorly-torch
!pip install -q fast-ctc-decode

import os
import sys
import time
import random
from datetime import datetime
from itertools import starmap
from time import perf_counter
from functools import partial
import numpy as np
import pandas as pd
import toml
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Module, ModuleList, Sequential, Conv1d, BatchNorm1d, Dropout, ReLU, SiLU
from torch.nn.functional import ctc_loss, log_softmax
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from google.colab import auth
from google.colab import drive as gdrive
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from bonito.nn import Permute
from fast_ctc_decode import beam_search, viterbi_search

# Tensor decomposition packages
import tensorly
from tltorch import FactorizedConv

# Models

In [ ]:
class Model(Module):
    """
    Model template for QuartzNet style architectures

    https://arxiv.org/pdf/1910.10261.pdf
    """

    def __init__(self, config):
        super(Model, self).__init__()
        if 'qscore' not in config:
            self.qbias = 0.0
            self.qscale = 1.0
        else:
            self.qbias = config['qscore']['bias']
            self.qscale = config['qscore']['scale']

        self.config = config
        self.stride = config['block'][0]['stride'][0]
        self.alphabet = config['labels']['labels']
        self.features = config['block'][-1]['filters']
        self.encoder = Encoder(config)
        self.decoder = Decoder(self.features, len(self.alphabet))

    def forward(self, x):
        encoded = self.encoder(x)
        return self.decoder(encoded)

    def decode(self, x, beamsize=5, threshold=1e-3, qscores=False, return_path=False):
        x = x.exp().cpu().numpy().astype(np.float32)
        if beamsize == 1 or qscores:
            seq, path  = viterbi_search(x, self.alphabet, qscores, self.qscale, self.qbias)
        else:
            seq, path = beam_search(x, self.alphabet, beamsize, threshold)
        if return_path: return seq, path
        return seq


class Encoder(Module):
    """
    Builds the model encoder
    """
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.config = config

        self.activations = {"relu": ReLU,"swish": SiLU}
        features = self.config['input']['features']
        activation = self.activations[self.config['encoder']['activation']]()
        encoder_layers = []

        for layer in self.config['block']:
            encoder_layers.append(
                Block(
                    features, layer['filters'], activation,
                    repeat=layer['repeat'], kernel_size=layer['kernel'],
                    stride=layer['stride'], dilation=layer['dilation'],
                    dropout=layer['dropout'], residual=layer['residual'],
                    separable=layer['separable'],
                )
            )

            features = layer['filters']

        self.encoder = Sequential(*encoder_layers)

    def forward(self, x):
        return self.encoder(x)


class TCSConv1d(Module):
    """
    Time-Channel Separable 1D Convolution
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False, separable=False):

        super(TCSConv1d, self).__init__()
        self.separable = separable

        if separable:
            # This layer cannot be factorised until "groups is implemented in tensorly-torch".
            self.depthwise = Conv1d(
                in_channels, in_channels, kernel_size=kernel_size, stride=stride,
                padding=padding, dilation=dilation, bias=bias, groups=in_channels
            )

            self.pointwise = Conv1d(
                in_channels, out_channels, kernel_size=1, stride=1,
                dilation=dilation, bias=bias, padding=0
            )

        else:
            self.conv = Conv1d(
                in_channels, out_channels, kernel_size=kernel_size,
                stride=stride, padding=padding, dilation=dilation, bias=bias
            )

    def forward(self, x):
        if self.separable:
            x = self.depthwise(x)
            x = self.pointwise(x)
        else:
            x = self.conv(x)
        return x


class Block(Module):
    """
    TCSConv, Batch Normalisation, Activation, Dropout
    """
    def __init__(self, in_channels, out_channels, activation, repeat=5, kernel_size=1, stride=1, dilation=1, dropout=0.0, residual=False, separable=False):

        super(Block, self).__init__()

        self.use_res = residual
        self.conv = ModuleList()

        _in_channels = in_channels
        padding = self.get_padding(kernel_size[0], stride[0], dilation[0])

        # add the first n - 1 convolutions + activation
        for _ in range(repeat - 1):
            self.conv.extend(
                self.get_tcs(
                    _in_channels, out_channels, kernel_size=kernel_size,
                    stride=stride, dilation=dilation,
                    padding=padding, separable=separable
                )
            )

            self.conv.extend(self.get_activation(activation, dropout))
            _in_channels = out_channels

        # add the last conv and batch norm
        self.conv.extend(
            self.get_tcs(
                _in_channels, out_channels,
                kernel_size=kernel_size,
                stride=stride, dilation=dilation,
                padding=padding, separable=separable
            )
        )

        # add the residual connection
        if self.use_res:
            self.residual = Sequential(*self.get_tcs(in_channels, out_channels))

        # add the activation and dropout
        self.activation = Sequential(*self.get_activation(activation, dropout))

    def get_activation(self, activation, dropout):
        return activation, Dropout(p=dropout)

    def get_padding(self, kernel_size, stride, dilation):
        if stride > 1 and dilation > 1:
            raise ValueError("Dilation and stride can not both be greater than 1")
        return (kernel_size // 2) * dilation

    def get_tcs(self, in_channels, out_channels, kernel_size=1, stride=1, dilation=1, padding=0, bias=False, separable=False):
        return [
            TCSConv1d(
                in_channels, out_channels, kernel_size,
                stride=stride, dilation=dilation, padding=padding,
                bias=bias, separable=separable
            ),
            BatchNorm1d(out_channels, eps=1e-3, momentum=0.1)
        ]

    def forward(self, x):
        _x = x
        for layer in self.conv:
            _x = layer(_x)
        if self.use_res:
            _x = _x + self.residual(x)
        return self.activation(_x)


class Decoder(Module):
    """
    Decoder
    """
    def __init__(self, features, classes):
        super(Decoder, self).__init__()
        self.layers = Sequential(
            Conv1d(features, classes, kernel_size=1, bias=True),
            Permute([2, 0, 1])
        )

    def forward(self, x):
        return log_softmax(self.layers(x), dim=2)

# Load config

In [ ]:
# Authenticate and create PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def download_toml_from_link(fn, link):
    _, id = link.split('=')
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile(fn)
    return toml.load(fn)

# Load model configuration

# dna_r9.4.1@v1 is heavier
#quartznet_config_link = "https://drive.google.com/open?id=1hKKE2Fzp3jdNyZI2h8jnOuwxBOvXWjp6"
#quartznet_config = download_toml_from_link("dna_r9.4.1@v1.toml",quartznet_config_link)

# dna_r9.4.1@v2 is lighter
quartznet_config_link = "https://drive.google.com/open?id=1IRDMrnE0WWeiRoioX7NHM5TXezl2jMkN"
quartznet_config = download_toml_from_link("dna_r9.4.1@v2.toml",quartznet_config_link)

# The structure of the model is defined using a config file.
# This will make sense to those familar with QuartzNet

print('Loaded quartznet config.')


Loaded quartznet config.


# Load state dict

In [ ]:

# mount users drive to load data
gdrive.mount('/content/drive', force_remount=True)

WEIGHTS_PATH = "/content/drive/MyDrive/Quartznet_weights/weights/last/weights_v2.pt"


model = Model(quartznet_config)

model.load_state_dict(torch.load(WEIGHTS_PATH))


Mounted at /content/drive


<All keys matched successfully>

# Create dict - INITIAL ASSIGNATION LOOP


In [ ]:
# PARAMETERS:
FACTORISATION = "TT" # tucker // CP // TT
RANK = 0.8

# Initialise convolutional doctionary
conv_layers_dict = dict()

# Main loop
for enc_dec_layer in model._modules.keys():

  print("[OUTER LAYER]", enc_dec_layer)

  if enc_dec_layer == "encoder":
    for layer in model.encoder.encoder._modules.keys():
      for sublayer in model.encoder.encoder[int(layer)]._modules.keys():

        print("--------------------------------", layer, sublayer)

        if sublayer == "conv":
          # add to this condicion: and int(layer) <=3 (<3 works)

          for index in  model.encoder.encoder[int(layer)].conv._modules.keys():

            if type(model.encoder.encoder[int(layer)].conv[int(index)]) == TCSConv1d:
              print("[DEBUG ------->>]", model.encoder.encoder[int(layer)].conv[int(index)])

              if "conv" in model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys():
                print(f"[DEBUG-PRE-conv] - layer: {layer} - sublayer: {sublayer} - index: {index} - current keys:", model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys())

                # Add to the dict
                conv_layers_dict[f"conv_{layer}{index}"] = [layer, index, "conv"]
                print(f"[DEBUG-DICT] - conv_{layer}{index}")

                # Factorisation
                layer_to_factor = model.encoder.encoder[int(layer)].conv[int(index)].conv
                rank = layer_to_factor.weight.size(0)//2
                model.encoder.encoder[int(layer)].conv[int(index)].conv = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)

                print("[DEBUG-POST]", type( model.encoder.encoder[int(layer)].conv[int(index)].conv))

              if "pointwise" in model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys():
                print(f"[DEBUG-PRE-pointwise] - layer: {layer} - sublayer: {sublayer} - index: {index} - current keys:", model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys())

                # Add to the dict
                conv_layers_dict[f"pointwise_{layer}{index}"] = [layer, index, "pointwise"]
                print(f"[DEBUG-DICT] - pointwise_{layer}{index}")

                # Factorisation
                layer_to_factor = model.encoder.encoder[int(layer)].conv[int(index)].pointwise
                rank = layer_to_factor.weight.size(0)//2
                model.encoder.encoder[int(layer)].conv[int(index)].pointwise = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)

                print("[DEBUG]", type( model.encoder.encoder[int(layer)].conv[int(index)].pointwise))

              if "depthwise" in model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys():
                print(f"[DEBUG-PRE-depthwise] - layer: {layer} - sublayer: {sublayer} - index: {index} - current keys:", model.encoder.encoder[int(layer)].conv[int(index)]._modules.keys())

                # Add to the dict
                conv_layers_dict[f"depthwise_{layer}{index}"] = [layer, index, "depthwise"]
                print(f"[DEBUG-DICT] - depthwise_{layer}{index}")

                # Factorisation needs to be removed as Tensorly-Torch has not implemented groups=in_channels, and the weights are not properly factorised
                """layer_to_factor = model.encoder.encoder[int(layer)].conv[int(index)].depthwise
                rank = layer_to_factor.weight.size(0)//2
                model.encoder.encoder[int(layer)].conv[int(index)].depthwise = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)"""

                print("[DEBUG]", type( model.encoder.encoder[int(layer)].conv[int(index)].depthwise))

        elif sublayer == "residual":
          print("[DEBUG ------->>]", model.encoder.encoder[int(layer)].residual)
          for index in  model.encoder.encoder[int(layer)].residual._modules.keys():

            if type(model.encoder.encoder[int(layer)].residual[int(index)]) == TCSConv1d:
              layer_to_factor = model.encoder.encoder[int(layer)].residual[int(index)].conv
              model.encoder.encoder[int(layer)].residual[int(index)].conv = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)
              print("[DEBUG]", type(model.encoder.encoder[int(layer)].residual[int(index)].conv))

  elif enc_dec_layer == "decoder":
     # Only one layer, hardcoded
        print("[DEBUG ------->>]", model.decoder.layers[0])
        layer_to_factor = model.decoder.layers[0]
        model.decoder.layers[0] = FactorizedConv.from_conv(layer_to_factor, rank=RANK, factorization=FACTORISATION, decompose_weights=False)
        print("[DEBUG]", type(model.decoder.layers[0]))


# DUMP / LOAD JSON

In [ ]:
import json
# mount users drive to save data
gdrive.mount('/content/drive', force_remount=True)
dict_savepath = '/content/drive/My Drive/Quartznet_weights/weights/json/' #@param {type:"string"}

# prevent overwriting of data
workdir = os.path.join(dict_savepath)

# Data to be written
with open(workdir+"layers.json", "w") as outfile:
    json.dump(conv_layers_dict, outfile)

Mounted at /content/drive


In [ ]:
import json
# mount users drive to save data
gdrive.mount('/content/drive', force_remount=True)
dict_savepath = '/content/drive/My Drive/Quartznet_weights/weights/json/' #@param {type:"string"}

# prevent overwriting of data
workdir = os.path.join(dict_savepath)

# Data to be written
with open(workdir+"layers.json", "r") as data:
    conv_dict = json.load(data)



Mounted at /content/drive


# SAVE COMPRESSED MODEL

In [1]:
# visualise model architecture
model.eval()

NameError: ignored

In [ ]:
# mount users drive to save data

gdrive.mount('/content/drive', force_remount=True)
compressed_weights_savepath = '/content/drive/My Drive/Quartznet_weights/weights/compressed/' #@param {type:"string"}

# prevent overwriting of data
workdir = os.path.join(compressed_weights_savepath, "weights_compressed_TT_v2.pt")

# Data to be written
torch.save(model.state_dict(), workdir)


Mounted at /content/drive


# Results: parameters

Import example from:
https://github.com/JeanKossaifi/tensorly-notebooks/blob/master/05_pytorch_backend/cnn_acceleration_tensorly_and_pytorch.ipynb

In [ ]:
# ---------------0.13--------------------

# Tucker (rank=1/8)
# v1
# Original: 6678533
# Compressed: 1414436

# v2
# Original: 6649613
# Compressed: 1256840

# -------------------------------------

# CP (rank=1/8)
# v1
# Original: 6678533
# Compressed: 1393154

# v2
# Original: 6649613
# Compressed: 1238177

# -------------------------------------

# TT (rank=1/8)
# v1
# Original: 6678533
# Compressed: 1393357

# v2
# Original: 6649613
# Compressed: 1245262

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

1245262

In [ ]:

# ---------------0.18--------------------

# Tucker (rank=1/6)
# v1
# Original: 6678533
# Compressed: 1686356

# v2
# Original: 6649613
# Compressed: 1542487
# -------------------------------------

# CP (rank=1/6)
# v1
# Original: 6678533
# Compressed: 1648459

# v2
# Original: 6649613
# Compressed: 1503985

# -------------------------------------

# TT (rank=1/6)
# v1
# Original: 6678533
# Compressed: 1646083

# v2
# Original: 6649613
# Compressed: 1496878

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

1496878

In [ ]:

# ---------------0.35--------------------

# Tucker (rank=1/3)
# v1
# Original: 6678533
# Compressed: 2777034

# v2
# Original: 6649613
# Compressed: 2657400
# -------------------------------------

# CP (rank=1/3)
# v1
# Original: 6678533
# Compressed: 2651531

# v2
# Original: 6649613
# Compressed: 2532753

# -------------------------------------

# TT (rank=1/3)
# v1
# Original: 6678533
# Compressed: 2649484

# v2
# Original: 6649613
# Compressed: 2532887

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

2532887

In [ ]:

# ---------------0.53--------------------

# Tucker (rank=1/2)
# v1
# Original: 6678533
# Compressed: 3907062

# v2
# Original: 6649613
# Compressed: 3829391
# -------------------------------------

# CP (rank=1/2)
# v1
# Original: 6678533
# Compressed: 3663306

# v2
# Original: 6649613
# Compressed: 3566653

# -------------------------------------

# TT (rank=1/2)
# v1
# Original: 6678533
# Compressed: 3659028

# v2
# Original: 6649613
# Compressed: 3555824

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

3555824

In [ ]:

# ---------------0.7--------------------

# Tucker (rank=0.7)
# v1
# Original: 6678533
# Compressed: 5296541

# v2
# Original: 6649613
# Compressed: 5222229
# -------------------------------------

# CP (rank=0.7)
# v1
# Original: 6678533
# Compressed: 4866478

# v2
# Original: 6649613
# Compressed: 4802220

# -------------------------------------

# TT (rank=0.7)
# v1
# Original: 6678533
# Compressed: 4876031

# v2
# Original: 6649613
# Compressed: 4801893

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

4801893

In [ ]:

# ---------------0.85--------------------

# Tucker (rank=0.8)
# v1
# Original: 6678533
# Compressed: 5973758

# v2
# Original: 6649613
# Compressed: 5922946
# -------------------------------------

# CP (rank=0.8)
# v1
# Original: 6678533
# Compressed: 5480530

# v2
# Original: 6649613
# Compressed: 5419782

# -------------------------------------

# TT (rank=0.8)
# v1
# Original: 6678533
# Compressed: 5468291

# v2
# Original: 6649613
# Compressed: 5416497

# -------------------------------------


np.sum([np.prod(p.size()) for p in model.parameters()])

5416497